In [1]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cleantext

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Niller\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Niller\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Storage: 
# define regular expressions for cleaning
num_re = re.compile(r"\d+")
# Formatting dates with YYYY-MM-DD since that's how it's formatted in the columns of the data.
date_re = re.compile(r"\d{4}-\d{2}-\d{2}")
email_re = re.compile(r"\S+@\S+")
url_re = re.compile(r"https?://\S+")
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
tokenlist = []
stopwordlist = []
stemmedlist = []
vocabulary_size = []
filtered_vocabulary_size = []
stemmed_vocabulary_size = []
def clean_text(text):
    # lower case all words
    text = text.lower()
    # replace multiple white spaces, tabs, or new lines with a single space
    text = re.sub(r"\s+", " ", text)
    # replace numbers, dates, emails, and URLs with special tokens
    text = date_re.sub("<DATE>", text)
    text = email_re.sub("<EMAIL>", text)
    text = url_re.sub("<URL>", text)
    text = num_re.sub("<NUM>", text)
    tokens = word_tokenize(text)
    # Removing tokens that come from tags
    tags = ['<','>','num','date','email','url']
    filtered_tokens = []
    for token in tokens:
        if token.lower() not in tags and re.match(r'^\w', token):
            filtered_tokens.append(token)
    # Removing stopwords
    tokenlist.append(filtered_tokens)
    stopwordless_tokens = []
    for token in filtered_tokens:
        if token.lower() not in stop_words:
            stopwordless_tokens.append(token)
    stopwordlist.append(stopwordless_tokens)
    # Perform stemming
    stemmed_tokens = [stemmer.stem(token) for token in stopwordless_tokens]
    stemmedlist.append(stemmed_tokens)
    # Compute vocabulary size before and after removing stopwords and stemming
    vocabulary_size.append(len(set(filtered_tokens)))
    filtered_vocabulary_size.append(len(set(stopwordless_tokens)))
    stemmed_vocabulary_size.append(len(set(stemmed_tokens)))
    stemmedtext = ' '.join(stemmed_tokens)
    return (stemmedtext)
#vocab_sum = sum(vocabulary_size)
#filtered_sum= sum(filtered_vocabulary_size)
#stemmed_sum = sum(stemmed_vocabulary_size)
#print("Vocabulary size:", vocab_sum ) 
#print("Vocabulary size without stopwords:",  filtered_sum, ", resulting in reduction rate:", filtered_sum/vocab_sum)
#print("Vocabulary size after stemming with no stopwords:", stemmed_sum, ", resulting in reduction rate:", stemmed_sum/filtered_sum)


In [2]:
df = pd.read_csv('news_sample.csv')
def clean_module(text):
    return cleantext.clean(text, clean_all= False, # Execute all cleaning operations
extra_spaces=True ,  # Remove extra white spaces 
lowercase=True ,# Convert to lowercase
numbers=True ,# Remove all digits 
punct=True ,# Remove all punctuations
stp_lang='english'
  # Language for stop words
)
def rem_stopwords(text):
    return cleantext.clean(text,stopwords=True)
def stem(text):
  return cleantext.clean(text,stemming=True)
def count_words(text):
    return len(text.split())
df['content'] = df['content'].apply(clean_module)
vocab_sum = df['content'].apply(count_words).sum
df['content'] = df['content'].apply(rem_stopwords)
filtered_sum= df['content'].apply(count_words).sum
df['content'] = df['content'].apply(stem)
stemmed_sum = df['content'].apply(count_words).sum
print("Vocabulary size:", vocab_sum ) 
print("Vocabulary size without stopwords:",  filtered_sum, ", resulting in reduction rate:", filtered_sum/vocab_sum)
print("Vocabulary size after stemming with no stopwords:", stemmed_sum, ", resulting in reduction rate:", stemmed_sum/filtered_sum)


Vocabulary size: <bound method NDFrame._add_numeric_operations.<locals>.sum of 0       505
1       176
2       451
3       504
4       283
       ... 
245    1292
246     574
247     220
248     424
249     111
Name: content, Length: 250, dtype: int64>


TypeError: unsupported operand type(s) for /: 'method' and 'method'

Observations: 

  1.All articles from www.newsmax.com, willyloman.wordpress and wallstreetonparade.com do not have a type.

2. The ids go up with the article order

3. The average amount of stemmed words change with each type, this is however a small sample size for a lot of the types.

4. The count of types vary a lot, 62% are "fake", yet only 1/250 is "clickbait" or "hate".

5. The words a more evenly spread without stopwords (see graphs below)


In [ ]:

from collections import Counter
import matplotlib.pyplot as plt
# Use Counter to count the frequency of each word
word_counts = Counter(sum(tokenlist, []))

# Get the 50 most common words
top_words = word_counts.most_common(50)

# Separate out the words and their counts
word_labels = [word[0] for word in top_words]
word_freqs = [word[1] for word in top_words]

import matplotlib.pyplot as plt
from collections import Counter

# Create the first subplot
plt.subplots(figsize=(25, 6))
plt.subplot(1, 2, 1)
plt.title('Top 50 words before processing')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.bar(word_labels, word_freqs)
plt.xticks(rotation=90)

# Create the second subplot
word_counts = Counter(sum(stopwordlist, []))
top_words = word_counts.most_common(50)
word_labels = [word[0] for word in top_words]
word_freqs = [word[1] for word in top_words]

plt.subplot(1, 2, 2)
plt.title('Top 50 words after removing stopwords')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.bar(word_labels, word_freqs)
plt.xticks(rotation=90)

# Show the figure
plt.subplots_adjust(wspace=0.15) # adjust the width space between subplots
plt.show()

In [ ]:
import pandas as pd

# Set the file path
file_path = 'news_cleaned_2018_02_13.csv'

# Determine the number of rows to skip based on the 10% sampling rate
num_rows = sum(1 for line in open(file_path, encoding='utf-8')) // 10

# Load the data, skipping every 10th row
dff = pd.read_csv(file_path, skiprows=lambda i: i % 10 != 0, encoding='utf-8')


In [ ]:
def clean_module(text):
    text = date_re.sub("<DATE>", text)
    return cleantext.clean(text, clean_all= False, # Execute all cleaning operations
extra_spaces=True ,  # Remove extra white spaces 
stemming=True , # Stem the words
stopwords=True ,# Remove stop words
lowercase=True ,# Convert to lowercase
numbers=True ,# Remove all digits 
punct=True ,# Remove all punctuations
stp_lang='english'
  # Language for stop words
)
# define regular expressions for cleaning
num_re = re.compile(r"\d+")
# Formatting dates with YYYY-MM-DD since that's how it's formatted in the columns of the data.
date_re = re.compile(r"\d{4}-\d{2}-\d{2}")
email_re = re.compile(r"\S+@\S+")
url_re = re.compile(r"https?://\S+")
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
tokenlist = []
stopwordlist = []
stemmedlist = []
vocabulary_size = []
filtered_vocabulary_size = []
stemmed_vocabulary_size = []
def clean_text(text):
    # lower case all words
    text = text.lower()
    # replace multiple white spaces, tabs, or new lines with a single space
    text = re.sub(r"\s+", " ", text)
    # replace numbers, dates, emails, and URLs with special tokens
    text = date_re.sub("<DATE>", text)
    text = email_re.sub("<EMAIL>", text)
    text = url_re.sub("<URL>", text)
    text = num_re.sub("<NUM>", text)
    tokens = word_tokenize(text)
    # Removing tokens that come from tags
    tags = ['<','>','num','date','email','url']
    filtered_tokens = []
    for token in tokens:
        if token.lower() not in tags and re.match(r'^\w', token):
            filtered_tokens.append(token)
    # Removing stopwords
    tokenlist.append(filtered_tokens)
    stopwordless_tokens = []
    for token in filtered_tokens:
        if token.lower() not in stop_words:
            stopwordless_tokens.append(token)
    stopwordlist.append(stopwordless_tokens)
    # Perform stemming
    stemmed_tokens = [stemmer.stem(token) for token in stopwordless_tokens]
    stemmedlist.append(stemmed_tokens)
    # Compute vocabulary size before and after removing stopwords and stemming
    vocabulary_size.append(len(set(filtered_tokens)))
    filtered_vocabulary_size.append(len(set(stopwordless_tokens)))
    stemmed_vocabulary_size.append(len(set(stemmed_tokens)))
    stemmedtext = ' '.join(stemmed_tokens)
    return (stemmedtext)

dff = dff.loc[dff['content'].apply(lambda x: not isinstance(x, float))]
dff['content'] = dff['content'].apply(clean_module)
#vocab_sum = sum(vocabulary_size)
#filtered_sum= sum(filtered_vocabulary_size)
#stemmed_sum = sum(stemmed_vocabulary_size)
#print("Vocabulary size:", vocab_sum ) 
#print("Vocabulary size without stopwords:",  filtered_sum, ", resulting in reduction rate:", filtered_sum/vocab_sum)
#print("Vocabulary size after stemming with no stopwords:", stemmed_sum, ", resulting in reduction rate:", stemmed_sum/filtered_sum)

print(dff)


In [ ]:
df.to_csv('10percentcleandata.csv', index=False)

In [3]:
df = pd.read_csv('10percentcleandata.csv')

C:\Users\Niller\AppData\Local\Temp\ipykernel_6028\876483081.py:1: DtypeWarning: Columns (0,1,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('10percentcleandata.csv')


In [ ]:

unique_values = df['type'].unique()

# print the unique values
print(unique_values)


In [ ]:
# create a dictionary to map the values to "fake" or "true"
mapping = {'rumor': 'fake',
           'hate': 'fake',
           'unreliable': 'fake',
           'conspiracy': 'fake',
           'clickbait': 'fake',
           'satire': 'fake',
           'bias': 'fake',
           'junksci': 'fake',
           'political': 'reliable'}

# replace the values using the mapping dictionary
df['type'] = df['type'].replace(mapping)
# filter out the rows where 'Column1' is not equal to either "true" or "fake"
df = df[(df['type'] == 'reliable') | (df['type'] == 'fake')]

In [ ]:

unique_values = df['type'].unique()

# print the unique values
print(unique_values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Create a CountVectorizer object to transform the documents into a bag of words representation
vectorizer = CountVectorizer()

# Fit the vectorizer to the documents and transform them into a matrix of word counts
wordvector = vectorizer.fit_transform(df['content'])

# Print the vocabulary of the vectorizer (the unique words in the corpus)
print(vectorizer.vocabulary_)

X_train, X_test, y_train, y_test = train_test_split(wordvector, df['type'], test_size=0.2)

# Train a logistic regression classifier on the training data
clf = LogisticRegression()

clf.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Create a CountVectorizer object to transform the documents into a bag of words representation
vectorizer = CountVectorizer()
# Create a new column in the dataframe for the domain information
#df['domain_info'] = df['domain'].astype(str)

# Concatenate the domain information with the content of each document
df['content'] = df['domain'] + df['content']
# Fit the vectorizer to the documents and transform them into a matrix of word counts
wordvector = vectorizer.fit_transform(df['content'])

# Print the vocabulary of the vectorizer (the unique words in the corpus)
print(vectorizer.vocabulary_)

X_train, X_test, y_train, y_test = train_test_split(wordvector, df['type'], test_size=0.2)

# Train a logistic regression classifier on the training data
clf = LogisticRegression()

clf.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#Data for advanced model
df = pd.read_csv('10percentcleandata.csv')
mapping = {'rumor': 'unreliable',
           'hate': 'fake',
           'unreliable': 'unreliable',
           'conspiracy': 'unreliable',
           'clickbait': 'unreliable',
           'satire': 'satire',
           'bias': 'unreliable',
           'junksci': 'fake',}
df = df[(df['type'] == 'reliable') | (df['type'] == 'fake')| (df['type'] == 'hate')| (df['type'] == 'satire')| (df['type'] == 'clickbait')| (df['type'] == 'rumor')]
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['type'], test_size=0.2)


We chose to split our articles into 4 types: "unreliable" (composed of "rumor", "conspiracy", "clickbait", "bias" and ofc itself), "fake" which is composed of "hate","junksci" and itself, "satire" which is only composed of itself and "reliable" which is only composed of itself. We chose to eliminate all the articles flagged as "political" as we don't see it fit in with any of the other categories since it by itself doesn't say anything about the truthfullness of the article.

In [ ]:
#Advanced model
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(df['content'])



In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


# Convert the text content into numerical data using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['content'])

# Encode the article type column as categorical variables
encoder = LabelEncoder()
y = encoder.fit_transform(df['type'])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the layers of the model
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer = Dense(64, activation='relu')(input_layer)
output_layer = Dense(len(encoder.classes_), activation='softmax')(hidden_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


KeyboardInterrupt: 